In [1]:
pip --version

pip 21.2.2 from C:\Users\1\anaconda3\envs\mypython3.6\lib\site-packages\pip (python 3.6)

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow 

In [3]:
print(tensorflow.version.VERSION)

1.15.0


In [4]:
import nltk
import sys
import numpy as np

nltk.download('brown')
nltk.download('universal_tagset')
data = nltk.corpus.brown.tagged_sents(tagset='universal')
all_tags = ['#EOS#','#UNK#','ADV', 'NOUN', 'ADP', 'PRON', 'DET', '.', 'PRT', 'VERB', 'X', 'NUM', 'CONJ', 'ADJ']

data = np.array([ [(word.lower(),tag) for word,tag in sentence] for sentence in data ])

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\1\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
C:\Users\1\anaconda3\envs\mypython3.6\lib\site-packages\ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
pip install -U scikit-learn scipy

In [5]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data,test_size=0.25,random_state=42)

In [6]:
from IPython.display import HTML, display
def draw(sentence):
    words,tags = zip(*sentence)
    display(HTML('<table><tr>{tags}</tr>{words}<tr></table>'.format(
                words = '<td>{}</td>'.format('</td><td>'.join(words)),
                tags = '<td>{}</td>'.format('</td><td>'.join(tags)))))
    
    
draw(data[11])
draw(data[10])
draw(data[7])

NOUN,ADP,NOUN,NOUN,NOUN,NOUN,VERB,ADV,VERB,ADP,DET,ADJ,NOUN,.,implementation,of,georgia's,automobile,title,law,was,also,recommended,by,the,outgoing,jury,.


PRON,VERB,ADP,DET,NOUN,.,VERB,NOUN,PRT,VERB,.,DET,NOUN,.,it,urged,that,the,city,``,take,steps,to,remedy,'',this,problem,.


NOUN,VERB,merger,proposed


In [7]:
from collections import Counter
word_counts = Counter()
for sentence in data:
    words,tags = zip(*sentence)
    word_counts.update(words)

all_words = ['#EOS#','#UNK#'] + list(list(zip(*word_counts.most_common(10000)))[0])

#let's measure what fraction of data words are in the dictionary
print("Coverage = %.5f" % (float(sum(word_counts[w] for w in all_words)) / sum(word_counts.values())))

Coverage = 0.92876


In [8]:
from collections import defaultdict
word_to_id = defaultdict(lambda:1, { word: i for i, word in enumerate(all_words) })
tag_to_id = { tag: i for i, tag in enumerate(all_tags)}

In [9]:
def to_matrix(lines, token_to_id, max_len=None, pad=0, dtype='int32', time_major=False):
    """Converts a list of names into rnn-digestable matrix with paddings added after the end"""
    
    max_len = max_len or max(map(len,lines))
    matrix = np.empty([len(lines), max_len],dtype)
    matrix.fill(pad)

    for i in range(len(lines)):
        line_ix = list(map(token_to_id.__getitem__,lines[i]))[:max_len]
        matrix[i,:len(line_ix)] = line_ix

    return matrix.T if time_major else matrix

In [10]:
batch_words, batch_tags = zip(*[zip(*sentence) for sentence in data[-3:]])

print("Word ids:")
print(to_matrix(batch_words, word_to_id))
print("Tag ids:")
print(to_matrix(batch_tags, tag_to_id))

Word ids:
[[   2 3057    5    2 2238 1334 4238 2454    3    6   19   26 1070   69
     8 2088    6    3    1    3  266   65  342    2    1    3    2  315
     1    9   87  216 3322   69 1558    4    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]
 [  45   12    8  511 8419    6   60 3246   39    2    1    1    3    2
   845    1    3    1    3   10 9910    2    1 3470    9   43    1    1
     3    6    2 1046  385   73 4562    3    9    2    1    1 3250    3
    12   10    2  861 5240   12    8 8936  121    1    4]
 [  33   64   26   12  445    7 7346    9    8 3337    3    1 2811    3
     2  463  572    2    1    1 1649   12    1    4    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]]
Tag ids:
[[ 6  3  4  6  3  3  9  9  7 12  4  5  9  4  6  3 12  7  9  7  9  8  4  6
   3  7  6 13  3  4  6  3  9  4  3  7  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0

In [14]:
pip install keras==2.0.8

  Attempting uninstall: keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
Note: you may need to restart the kernel to use updated packages.


In [15]:
import keras
import keras.layers as L

model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.SimpleRNN(64,return_sequences=True))

#add top layer that predicts tag probabilities
stepwise_dense = L.Dense(len(all_tags),activation='softmax')
stepwise_dense = L.TimeDistributed(stepwise_dense)
model.add(stepwise_dense)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [16]:
from keras.utils.np_utils import to_categorical
BATCH_SIZE=32
def generate_batches(sentences,batch_size=BATCH_SIZE,max_len=None,pad=0):
    assert isinstance(sentences,np.ndarray),"Make sure sentences is q numpy array"
    
    while True:
        indices = np.random.permutation(np.arange(len(sentences)))
        for start in range(0,len(indices)-1,batch_size):
            batch_indices = indices[start:start+batch_size]
            batch_words,batch_tags = [],[]
            for sent in sentences[batch_indices]:
                words,tags = zip(*sent)
                batch_words.append(words)
                batch_tags.append(tags)

            batch_words = to_matrix(batch_words,word_to_id,max_len,pad)
            batch_tags = to_matrix(batch_tags,tag_to_id,max_len,pad)

            batch_tags_1hot = to_categorical(batch_tags,len(all_tags)).reshape(batch_tags.shape+(-1,))
            yield batch_words,batch_tags_1hot

In [17]:
def compute_test_accuracy(model):
    test_words,test_tags = zip(*[zip(*sentence) for sentence in test_data])
    test_words,test_tags = to_matrix(test_words,word_to_id),to_matrix(test_tags,tag_to_id)

    #predict tag probabilities of shape [batch,time,n_tags]
    predicted_tag_probabilities = model.predict(test_words,verbose=1)
    predicted_tags = predicted_tag_probabilities.argmax(axis=-1)

    #compute accurary excluding padding
    numerator = np.sum(np.logical_and((predicted_tags == test_tags),(test_words != 0)))
    denominator = np.sum(test_words != 0)
    return float(numerator)/denominator


class EvaluateAccuracy(keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        sys.stdout.flush()
        print("\nMeasuring validation accuracy...")
        acc = compute_test_accuracy(self.model)
        print("\nValidation accuracy: %.5f\n"%acc)
        sys.stdout.flush()

In [18]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/5




1341/1343 [============================>.] - ETA: 0s - loss: 0.2530
Measuring validation accuracy...
14208/14335 [============================>.] - ETA: 0s
Validation accuracy: 0.94080

1344/1343 [==============================] - 33s - loss: 0.2526    
Epoch 2/5
1342/1343 [============================>.] - ETA: 0s - loss: 0.0576
Measuring validation accuracy...
14272/14335 [============================>.] - ETA: 0s
Validation accuracy: 0.94505

1344/1343 [==============================] - 32s - loss: 0.0576    
Epoch 3/5
1341/1343 [============================>.] - ETA: 0s - loss: 0.0507
Measuring validation accuracy...
14208/14335 [============================>.] - ETA: 0s
Validation accuracy: 0.94584

1344/1343 [==============================] - 32s - loss

In [19]:
acc = compute_test_accuracy(model)
print("Final accuracy: %.5f"%acc)

assert acc>0.94, "Keras has gone on a rampage again, please contact course staff."

14335/14335 [==============================] - 7s     
Final accuracy: 0.94536


In [20]:
#Define a model that utilizes bidirectional SimpleRNN
model = keras.models.Sequential()
model.add(L.InputLayer([None],dtype='int32'))
model.add(L.Embedding(len(all_words),50))
model.add(L.Bidirectional(L.LSTM(64, return_sequences=True)))
model.add(L.Bidirectional(L.LSTM(32, return_sequences=True)))
model.add(L.Dense(len(all_tags),activation='softmax'))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [21]:
model.compile('adam','categorical_crossentropy')

model.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,)

Epoch 1/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.2836
Measuring validation accuracy...
14335/14335 [==============================] - 43s    

Validation accuracy: 0.95409

1344/1343 [==============================] - 322s - loss: 0.2834   
Epoch 2/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0457
Measuring validation accuracy...
14335/14335 [==============================] - 37s    

Validation accuracy: 0.96187

1344/1343 [==============================] - 221s - loss: 0.0457   
Epoch 3/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0374
Measuring validation accuracy...
14335/14335 [==============================] - 39s    

Validation accuracy: 0.96374

1344/1343 [==============================] - 224s - loss: 0.0374   
Epoch 4/5
1343/1343 [============================>.] - ETA: 0s - loss: 0.0319
Measuring validation accuracy...
14335/14335 [==============================] - 38s    

Validation accuracy: 0.96505

1344/134

In [22]:
acc = compute_test_accuracy(model)
print("\nFinal accuracy: %.5f"%acc)

assert acc>0.96, "Bidirectional RNNs are better than this!"
print("Well done!")

14335/14335 [==============================] - 44s    

Final accuracy: 0.96497
Well done!


In [23]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to c:\users\1\appdata\local\temp\pip-req-build-d5qfmem9
Note: you may need to restart the kernel to use updated packages.


  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git 'C:\Users\1\AppData\Local\Temp\pip-req-build-d5qfmem9'


  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101669 sha256=5b372ff21469bfbed6753b644442f4b9a177a30d790341bce119c316add6bc59
  Stored in directory: C:\Users\1\AppData\Local\Temp\pip-ephem-wheel-cache-dx2bj1c0\wheels\16\87\6e\8e3b73f23fb38163af1c319aa23f14602018b501ecb91430a2
Successfully built keras-contrib


In [24]:
from keras_contrib.layers import CRF

In [27]:
model2 = keras.models.Sequential()
model2.add(L.InputLayer([None],dtype='int32'))
model2.add(L.Embedding(len(all_words),50))
model2.add(L.Bidirectional(L.LSTM(64, return_sequences=True)))
model2.add(L.Bidirectional(L.LSTM(32, return_sequences=True)))
model2.add(L.Dense(len(all_tags),activation='softmax'))
crf_ = CRF(units=len(all_words), sparse_target=True)
model2.add(crf_)

In [28]:
model2.compile('adam', loss = crf_.loss_function, metrics=[crf_.accuracy])

model2.fit_generator(generate_batches(train_data),len(train_data)/BATCH_SIZE,
                    callbacks=[EvaluateAccuracy()], epochs=5,) 

AttributeError: 'CRF' object has no attribute '_outbound_nodes'